In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Load Model & Tokenizer
model = load_model('halal_haram_lstm_finetuned_model.h5')
with open('tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

# Inisialisasi Lemmatizer & Stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# Function: Preprocess Text for Inference
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Function: Predict Halal or Haram with Confidence Score
def predict_label_with_confidence(text):
    text = clean_text(text)  # Preprocess text
    sequence = tokenizer.texts_to_sequences([text])  # Tokenize
    padded_sequence = pad_sequences(sequence, maxlen=100, padding='post')  # Padding
    prediction = model.predict(padded_sequence)[0][0]  # Probabilitas sigmoid

    confidence = round(prediction * 100, 2)  # Konversi ke persen
    label = "Halal" if prediction >= 0.3 else "Haram"
    
    return f"{label} ({confidence}%)"

# Contoh Inference
new_texts = [
    "beef stock contains yeast extract and flavor enhancer",
    "pure vegetable oil with no additives",
    "contains pork gelatin and artificial coloring",
    "clam stock potatoes clams cream vegetable oil corn canola andor soybean celery contains less than of modified food starch salt wheat flour soy protein concentrate uncured smoked bacon piecesno nitrates or nitrites added except for those naturally occurring in sea salt pork water sea salt sugar dried onions spicessodium phosphate flavoring clam extract succinic acid sugar soy lecithin soy sauce soybeans wheat saltcontains cod wheat milk soy",
    "water cream broccoli celery vegetable oil corn canola andor soybean modified food starch cheddar cheese cheddar cheese cultured milk salt enzymes whey salt sodium phosphate contains less than of butter parmesan and cheddar cheese milk cultures salt enzymes wheat flour salt potatoes onions soy protein concentrate roasted garlic annatto extract for color soy lecithin driedcontains wheat milk soy",
    "prepared organic chick peas water sea salt"
]

for text in new_texts:
    print(f"Text: {text} → Prediction: {predict_label_with_confidence(text)}")


1/1 [==============================] - 0s 135ms/step
Text: beef stock contains yeast extract and flavor enhancer → Prediction: Haram (2.72%)
1/1 [==============================] - 0s 9ms/step
Text: pure vegetable oil with no additives → Prediction: Halal (99.35%)
1/1 [==============================] - 0s 10ms/step
Text: contains pork gelatin and artificial coloring → Prediction: Haram (0.01%)
1/1 [==============================] - 0s 9ms/step
Text: clam stock potatoes clams cream vegetable oil corn canola andor soybean celery contains less than of modified food starch salt wheat flour soy protein concentrate uncured smoked bacon piecesno nitrates or nitrites added except for those naturally occurring in sea salt pork water sea salt sugar dried onions spicessodium phosphate flavoring clam extract succinic acid sugar soy lecithin soy sauce soybeans wheat saltcontains cod wheat milk soy → Prediction: Haram (0.31%)
1/1 [==============================] - 0s 9ms/step
Text: water cream brocco